https://starpentagon.net/analytics/hypernyms_of_wordnet_id/

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download()

In [6]:
synset=wn.synset_from_pos_and_offset('n', 2110341)
synset,synset.hypernyms()

(Synset('dalmatian.n.02'), [Synset('dog.n.01')])

https://qiita.com/pocket_kyoto/items/f2382b9821d32624e6bc

In [1]:
import sqlite3
conn = sqlite3.connect("wnjpn.db")

class node:
    def __init__(self, name, children=None):
        self.name = name  # String
        self.children = children  # List of Class node

    # 結果表示用
    def display(self, indent = 0):
        if self.children != None:
            print(' '*indent + self.name)
            for c in self.children:
                c.display(indent+1)
        else:
            print(' '*indent + self.name)

In [2]:
conn = sqlite3.connect("wnjpn.db")
# 含まれるテーブルの確認
cur = conn.execute("select name from sqlite_master where type='table'")
for row in cur:
    print(row)

('pos_def',)
('link_def',)
('synset_def',)
('synset_ex',)
('synset',)
('synlink',)
('ancestor',)
('sense',)
('word',)
('variant',)
('xlink',)


In [3]:
# 上位-下位の関係にある概念の抽出
hierarchy_dict = {}  # key:上位語(String), value:下位語(List of String)
n_term_set = set()  # 下位語に含まれる単語集合

cur = conn.execute("select synset1,synset2 from synlink where link='hypo'")  # 上位語-下位語の関係にあるものを抽出
for row in cur:
    b_term = row[0]
    n_term = row[1]

    if b_term not in hierarchy_dict:
        hierarchy_dict[b_term] = []

    hierarchy_dict[b_term].append(n_term) 
    n_term_set.add(n_term)

print("上位語に含まれる単語の数 ： %s" % len(hierarchy_dict))

top_concepts = list(set(hierarchy_dict.keys()) - n_term_set)
print("上位語に含まれる単語の中で下位語に含まれない単語の数 ： %s" % len(top_concepts))

上位語に含まれる単語の数 ： 20008
上位語に含まれる単語の中で下位語に含まれない単語の数 ： 346


In [4]:
# synset(概念)のIDから、概念の名称に変換する辞書の作成
synset_name_dict = {}  # key:synsetのID, value:synsetの名称
cur = conn.execute("select synset,name from synset")
for row in cur:
    synset_name_dict[row[0]] = row[1]
for k,v in synset_name_dict.items():
    print("%s : %s" % (k,v))
    break

07125096-n : expletive


In [5]:
# 下位の語が未登録の場合、再帰的に呼ばれる関数
def recrusive_register(term, synset_name_dict, hierarchy_dict, node_tree_dict):
    for term in hierarchy_dict[term]:
        if term not in node_tree_dict:
            if term in hierarchy_dict:
                recrusive_register(term, synset_name_dict, hierarchy_dict, node_tree_dict)
                node_tree_dict[term] = node(synset_name_dict[term], [node_tree_dict[t] for t in hierarchy_dict[term]])
            else:
                node_tree_dict[term] = node(synset_name_dict[term])

# データ投入
node_tree_dict = {}
for k in top_concepts:  # 最上位の語を起点として木構造を作成
    recrusive_register(k, synset_name_dict, hierarchy_dict, node_tree_dict)
    node_tree_dict[k] = node(synset_name_dict[k], [node_tree_dict[term] for term in hierarchy_dict[k]])


In [6]:
node_tree_dict[top_concepts[10]].display()

come
 set_down
  alight
  force-land
  beach
  port
  set_down
  touch_down
  undershoot
  belly-land
  crash_land
   ditch
 drive_in
 put_down
 land
 roll_up
 come
 hit
  max_out
  break_even
 flood_in
 pull_in
 plump_in


In [7]:
# 下位-上位の関係にある概念の抽出
cur = conn.execute("select synset1,synset2 from synlink where link='hypo'")

hierarchy_dict = {}  # key:下位語(String), value:上位語(String)

for row in cur:
    b_term = row[0]
    n_term = row[1]

    if n_term not in hierarchy_dict:
        hierarchy_dict[n_term] = b_term

In [8]:
# 特定の単語を入力とした時に、上位語を検索する関数
def SearchTopConceptWords(word, hierarchy_dict):

    # 問い合わせしたい単語がWordnetに存在するか確認する
    cur = conn.execute("select wordid from word where lemma='%s'" % word)
    word_id = 99999999  #temp 
    for row in cur:
        word_id = row[0]

    # Wordnetに存在する語であるかの判定
    if word_id==99999999:
        print("「%s」は、Wordnetに存在しない単語です。" % word)
        return
    else:
        print("【「%s」の最上位概念を出力します】\n" % word)

    # 入力された単語を含む概念を検索する
    cur = conn.execute("select synset from sense where wordid='%s'" % word_id)
    synsets = []
    for row in cur:
        synsets.append(row[0])

    # 概念に含まれる単語を検索して画面出力する
    no = 1
    for synset in synsets:
        cur1 = conn.execute("select name from synset where synset='%s'" % synset)
        for row1 in cur1:
            print("%sつめの概念 : %s" %(no, row1[0]))
        cur2 = conn.execute("select def from synset_def where (synset='%s' and lang='jpn')" % synset)
        sub_no = 1
        for row2 in cur2:
            print("意味%s : %s" %(sub_no, row2[0]))
            sub_no += 1

        # 上位語の検索部分
        b_term = ""
        while(synset in hierarchy_dict.keys()):
            synset = hierarchy_dict[synset]

        cur1 = conn.execute("select name from synset where synset='%s'" % synset)
        for row1 in cur1:
            print("最上位概念 : %s" % row1[0])

        cur2 = conn.execute("select def from synset_def where (synset='%s' and lang='jpn')" % synset)
        sub_no = 1
        for row2 in cur2:
            print("意味%s : %s" %(sub_no, row2[0]))
            sub_no += 1

        # 更新          
        print("\n")
        no += 1

In [9]:
SearchTopConceptWords("ネコ", hierarchy_dict)

【「ネコ」の最上位概念を出力します】

1つめの概念 : true_cat
意味1 : 通常、厚く柔らかい毛皮を持ち、吠えることのできないネコ科の哺乳類：家ネコ
意味2 : ヤマネコ
最上位概念 : entity
意味1 : （生命がある、あるいは生命がないに関わらず）それ自身の明確な存在を持つと感知される、知られている、あるいは推定される何か


